# RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.

# [RAGAS](https://www.ragas.io/)
- RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크이다. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas rapidfuzz`

In [1]:
!uv pip install ragas rapidfuzz

Resolved 102 packages in 1.71s
Prepared 14 packages in 632ms
Uninstalled 2 packages in 47ms
Installed 15 packages in 1.61s
 + appdirs==1.4.4
 + cfgv==3.5.0
 + datasets==4.4.2
 + diskcache==5.6.3
 + distlib==0.4.0
 - fsspec==2025.12.0
 + fsspec==2025.10.0
 + identify==2.6.15
 + instructor==1.13.0
 - jiter==0.12.0
 + jiter==0.11.1
 + nodeenv==1.10.0
 + pre-commit==4.5.1
 + ragas==0.4.2
 + scikit-network==0.33.5
 + ty==0.0.7
 + virtualenv==20.35.4


## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해서 생성되었는지 평가한다. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인한다.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단한다. 이를 바탕으로 faithfulness 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산한다. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용한다.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가한다.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input) 간의 유사도를 측정한다.

#### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성한다.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정한다. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

## Retrieval 평가
Vector store에서 검색한 context에 대한 평가

### Context Precision
- 검색된 문서(context)들 중 질문과 관련 있는 것들이 얼마나 **상위 순위**에 있는 지 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


#### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{Precision@k}$: 개별 문서에 대한 Precision
- K: context 의 개수(chuck 수)
- $v_k$: 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

#### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context
        - 한국의 수도는 서울이고 인구는 5000명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
    ```bash
    Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  1/1(1번 문서 계산 시에는 1개 문서만 있으므로 분모가 1이 된다.)
    Precision@2 = 1/2 = 0.5     
    Precision@3 = 2/3 ≈ 0.67    
    Precision@4 = 3/4 = 0.75
    Precision@5 = 3/5 = 0.6
    ```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

# GUIDE
- 예를 들어, 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정하면:
```bash
Precision@1 = 1/1 = 1.0     # 1등 -> 1/1  --> 1 등이 관련 없는 것이면 전체 평균 점수가 확 낮아 질 수있다.
Precision@2 = 1/2 = 0.5     # 2등 -> 1/2  (1, 2등 두개 중에 하나만 관련(1번)이 있다.)
Precision@3 = 2/3 ≈ 0.67    # 3등 -> 2/3  (1, 2, 3 등 세개 중에 두개만 관련(1,3번)이 있다.)
Precision@4 = 3/4 = 0.75
Precision@5 = 3/5 = 0.
```
왜 분모가 1, 2, 3, 4, 5 -> 그 순위에는 상위순위와 자신밖에 없으므로 그 개수가 분모가됨. (3등은 1, 2, 3 등 세개 문서로.)

### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

#### 평가방법
1. 정답에서 주장(claim)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장(claim)의 정보를 검색된 contexts에서 찾을 수 있는지 판별한다. 이를 바탕으로 context recall 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

# RAGAS 평가 실습

In [ ]:
# !uv pip install ragas rapidfuzz
# 설치 후 커널 재시작

In [1]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, SparseVectorParams, VectorParams
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()


True

In [2]:
# ##############################################################
# 데이터 준비
##############################################################

def load_and_split_olympic_data(file_path="data/olympic_wiki.md"):
    with open(file_path, "r", encoding="utf-8") as fr:
        olympic_text = fr.read()

    # Split
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ],
        # strip_headers=False, # 문서에 header 포함 여부(default: True - 제거)
    )

    return splitter.split_text(olympic_text)

In [3]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str = "olympic_info_wiki"):


    dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    client = QdrantClient(host="localhost", port=6333)

    # 컬렉션 삭제
    if client.collection_exists(collection_name):
        result = client.delete_collection(collection_name=collection_name)

    # 컬렉션 생성
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
    )

    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,    
        embedding=dense_embeddings
    )
    
    ######################################
    # Document들 추가
    ######################################
    documents = load_and_split_olympic_data()
    vectorstore.add_documents(documents=documents)

    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [4]:
vectorstore = get_vectorstore()

retriever = get_retriever(vectorstore)
retriever

VectorStoreRetriever(tags=['QdrantVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_qdrant.qdrant.QdrantVectorStore object at 0x000001B8AD1F2060>, search_kwargs={'k': 5})

In [ ]:
################################################################################
# 평가할 RAG Chain
################################################################################

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


vectorstore = get_vectorstore()
retriever = get_retriever(vectorstore)

prompt_txt = """# Instruction:
당신은 정보제공을 목적으로하는 유능한 AI Assistant 입니다.
주어진 context의 내용을 기반으로 질문에 답변을 합니다.
Context에 질문에 대한 명확한 정보가 있는 경우 그것을 바탕으로 답변을 합니다.
Context에 질문에 대한 명확한 정보가 없는 경우 "정보가 부족해 답을 할 수없습니다." 라고 답합니다.
절대 추측이나 일반 상식을 바탕으로 답을 하거나 Context 없는 내용을 만들어서 답변해서는 안됩니다.

# Context:
{context}

# 질문:
{query}
"""
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# chain = {
#     "context":retriever | format_docs,
#     "query":RunnablePassthrough()
# } | prompt | model | parser


# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경.
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict: dict) -> list:
    # dictionary[list[Document], query:str] -> list[str]  
    # 문서내용만 추출해서(Document.page_content)만 추출한 리스트
    return [doc.page_content for doc in docs_dict['context']]

# dict | dict -> dict
# RunnablePassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    "response": prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가시 context -> List[str]
}

In [41]:
res = chain.invoke("1회 올림픽은 언제 어디서 열렸지")

In [42]:
res.keys()

dict_keys(['response', 'retrieved_context'])

In [43]:
print(res['response'])

제1회 올림픽은 1896년에 그리스 아테네에서 열렸습니다.


In [45]:
res['retrieved_context']

['고대의 올림픽 경기(올림피아 경기)는 고대 그리스의 여러 도시 국가의 대표선수들이 모여 벌인 일련의 시합이었으며, 육상 경기가 주 종목이지만 격투기와 전차 경기도 열렸다. 그리고 패배하면 죽기도 하였다. 고대 올림픽의 유래는 수수께끼로 남아있다. 잘 알려진 신화로는 헤라클레스와 그의 아버지인 제우스가 올림픽의 창시자였다는 것이다. 전설에 따르면 이 경기를 최초로 \'올림픽\'이라고 부르고, 4년마다 대회를 개최하는 관례를 만든 사람이 헤라클레스라고 한다. 어떤 전설에서는 헤라클레스가 이른바 헤라클레스의 12업을 달성한 뒤에 제우스를 기리고자 올림픽 경기장을 지었다고 한다. 경기장이 완성되자 헤라클레스는 일직선으로 200 걸음을 걸었으며, 이 거리를 "스타디온"이라 불렀는데, 후에 이것이 길이 단위인 \'스타디온\'(그리스어: στάδιον → 라틴어: 영어: stadium)이 되었다. 또 다른 설로는 \'올림픽 휴전\'(그리스어: ἐκεχειρία 에케케이리아[*])이라는 고대 그리스의 관념이 최초의 올림피아 경기와 관련이 있다고 한다. \'올림픽 휴전\'이란 어느 도시 국가라도 올림피아 경기 기간 중에 다른 나라를 침범하면 그에 대한 응징을 받을 수 있다는 뜻으로, "올림픽 기간에는 전쟁하지 말 것"으로 요약할 수 있다.  \n고대 올림피아 경기가 처음 열린 시점은 보통 기원전 776년으로 인정되고 있는데, 이 연대는 그리스 올림피아에서 발견된 비문에 근거를 둔 것이다. 이 비문의 내용은 달리기 경주 승자 목록이며 기원전 776년부터 4년 이후 올림피아 경기 마다의 기록이 남겨져 있다. 고대 올림픽의 종목으로는 육상, 5종 경기(원반던지기, 창던지기, 달리기, 레슬링, 멀리뛰기), 복싱, 레슬링, 승마 경기가 있었다. 전설에 따르면 엘리스의 코로이보스가 최초로 올림피아 경기에서 우승한 사람이라고 한다.  \n고대 올림피아 경기는 근본적으로 종교적인 중요성을 띄고 있었는데, 스포츠 경기를 할 때는 제우스(올림피아의 제우스 신전에는 페이디아스가 만든 제우스 

# RAGAS 를 이용해 평가를 위한 합성 데이터 셋 만들기

- 평가 데이터셋 구성
  - `user_input`: 사용자 질문
  - `retrieved_contexts`: Vectorstore에서 검색한 context
  - `response`: LLM의 응답
  - `reference`: 정답

## TestsetGenerator
- **문서(retrieved_contexts)를 기준**으로 **질문**, **정답** 을 생성한다.
- 평가할 LLM으로 생성된 질문을 넣어 답변을 추출하여 데이터셋을 구성한다.

In [ ]:
# 주피터노트북 환경에서 비동기적 처리 위해
# script(.py) 로 작성할 경우는 필요 없다.

import nest_asyncio
nest_asyncio.apply()

In [17]:
###########################################
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
###########################################
client = QdrantClient(host="localhost", port=6333)
COLLECTION_NAME = "olympic_info_wiki"

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(COLLECTION_NAME)
# info.points_count # 저장된 데이터 개수
results, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=info.points_count
)

# sampling
import random
sample_dataset = random.sample(results, 5) # 리스트에서 랜덤하게 K개를 추출

# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

['올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 상업성과 연계되어 이제 올림픽이 기존의 상업적인 스포츠쇼와 다를 게 없다는 것이다. 1996년과 2000년 하계 올림픽 기간 사이에는 올림픽 관련 상품 시장의 포화 상태가 일어나면서 IOC에게 또 다른 비판이 일었다. 개최도시가 올림픽 관련 물건들을 파려는 상인과 회사들로 넘쳐났던 것이다. IOC는 이에 대해 앞으로 다가올 올림픽에서는 과다 경쟁을 방지하고 이런 불미스라운 상황이 일어나지 않도록 대처하겠다고 응답했다. 또 다른 비판으로는, 개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있다는 점이다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 한다.',
 "국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을 받고 있다. 그 예로 IOC 위원장이었던 에이버리 브런디지와 후안 안토니오 사마란치가 대표적인 사람이다. 브런디지는 20년 넘게 IOC 위원장직을 맡았고 임기 중에 올림픽을 정치적으로 휘말려들지 않게 하기 위해 보호했다. 그러나 그는 남아프리카 공화국 대표단에게 아파르트헤이트와 관련된 이슈를 건드리고 반유대정책을 함으로써 비난을 받았다. 사마란치 위원장 시기 때는 족벌 정치와 부패로 비난받았다. 사마란치가 스페인에서 프랑코 정권에 협력했다는 것도 비판의 이유가 되었다.  \n1998년에 몇몇 IOC위원들이 2002년 솔트레이크 시티 동계 올림픽 유치 과정에서 미국에게 미국을 올림픽 개최지로 뽑아달라는 뇌물청탁을 받았다는 것이 폭로되었다. 이에 IOC는 사퇴한 IOC위원 4명과 강제 퇴출된 6명에 대한 조사를 했다. 이 스캔들은 이후에 개최지 선정에서 이와 같은 불미스러운 일이 일어

In [12]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# gpt-5 버전은 사용할 수없다. (temperature 설정 문제가 있다.)
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1")) #Langchain모델을 RAGAS에서 사용할 수있도록 변환(Wrapping)
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context="데이터셋은 반드시 한국어로 작성한다. 데이터셋은 JSON 문법을 꼭 지켜서 작성한다.\
특히 구두점은 꼭 지켜야 한다. Document에 JSON 문법에 맞지 않는 표현이 있으면 반드시 수정해서 처리한다."  
    # 질문/답변 생성할 때 llm에 전달할 추가 system prompt를 설정.
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_23072\1878484181.py:8: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1")) #Langchain모델을 RAGAS에서 사용할 수있도록 변환(Wrapping)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23072\1878484181.py:9: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))


In [19]:
testset = generator.generate_with_chunks(
    docs, testset_size=10  # context 내용, 테스트데이터셋 몇개를 만들지.
)

Applying SummaryExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

Node 1d2d45a1-ba9e-4fb3-92dd-9abfd1a36c16 does not have a summary. Skipping filtering.
Node 32f07a06-e0de-4660-b44a-538aa95a9118 does not have a summary. Skipping filtering.
Node 5c00707e-1745-4d48-87a2-7426b4519206 does not have a summary. Skipping filtering.
Node e2e3a4a5-d503-4adb-a9cd-108b9020d66a does not have a summary. Skipping filtering.
Node 9077469d-0511-4239-9774-2c6556fc256a does not have a summary. Skipping filtering.


Applying EmbeddingExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying ThemesExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

...

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.


Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
print(type(testset))
testset

<class 'ragas.testset.synthesizers.testset_schema.Testset'>


Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='개최국이 올림픽 개최할 때 돈 많이 쓰고 IOC는 그냥 앉아서 올림픽 상징으로 돈 다 가져가고, 개최국이나 개최도시는 투자한 만큼 이익 볼지 확신도 못하면서도 계속 노력해야 하는 거 왜 그런지 설명해 주세요.', retrieved_contexts=None, reference_contexts=['올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 상업성과 연계되어 이제 올림픽이 기존의 상업적인 스포츠쇼와 다를 게 없다는 것이다. 1996년과 2000년 하계 올림픽 기간 사이에는 올림픽 관련 상품 시장의 포화 상태가 일어나면서 IOC에게 또 다른 비판이 일었다. 개최도시가 올림픽 관련 물건들을 파려는 상인과 회사들로 넘쳐났던 것이다. IOC는 이에 대해 앞으로 다가올 올림픽에서는 과다 경쟁을 방지하고 이런 불미스라운 상황이 일어나지 않도록 대처하겠다고 응답했다. 또 다른 비판으로는, 개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있다는 점이다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 한다.'], retrieved_context_ids=None, reference_context_ids=None, response=None, multi_responses=None, reference='개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있습니다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져갑니다. 개최도시 입장에서는

In [ ]:
sample1  = testset.samples[0].eval_sample
print(sample1.user_input)         # 사용자 질문
print(sample1.reference_contexts) # Vector DB의 context (이것을 바탕으로 user_input과 referenece를 생성)
print(sample1.retrieved_contexts) # RAG pipeline이 벡터디비에서 검색한 문서
print(sample1.reference)          # 정답(ground truth)
print(sample1.response)           # LLM 응답 (정답추정값)

# 평가시 필요한 속성: user_input, retrieved_contexts, reference, response
#   retrieved_context, response -> 평가할 RAG Chain으로 부터 가져와야 한다.

개최국이 올림픽 개최할 때 돈 많이 쓰고 IOC는 그냥 앉아서 올림픽 상징으로 돈 다 가져가고, 개최국이나 개최도시는 투자한 만큼 이익 볼지 확신도 못하면서도 계속 노력해야 하는 거 왜 그런지 설명해 주세요.
['올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 상업성과 연계되어 이제 올림픽이 기존의 상업적인 스포츠쇼와 다를 게 없다는 것이다. 1996년과 2000년 하계 올림픽 기간 사이에는 올림픽 관련 상품 시장의 포화 상태가 일어나면서 IOC에게 또 다른 비판이 일었다. 개최도시가 올림픽 관련 물건들을 파려는 상인과 회사들로 넘쳐났던 것이다. IOC는 이에 대해 앞으로 다가올 올림픽에서는 과다 경쟁을 방지하고 이런 불미스라운 상황이 일어나지 않도록 대처하겠다고 응답했다. 또 다른 비판으로는, 개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있다는 점이다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 한다.']
None
개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있습니다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져갑니다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 합니다.
None


In [27]:
# Testset을 pandas DataFrame을 변환 -> to_pandas(), to_xxxx() 를 통해서 다른 구조로 변환이 가능.
eval_df = testset.to_pandas()
eval_df

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,개최국이 올림픽 개최할 때 돈 많이 쓰고 IOC는 그냥 앉아서 올림픽 상징으로 돈 ...,[올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 ...,개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽...,Sports Historian,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
1,"프랑코 정권이 뭐였는지, 그리고 사마란치가 프랑코 정권에 협력한 게 왜 IOC에서 ...",[국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을...,사마란치 위원장은 스페인에서 프랑코 정권에 협력했다는 이유로 비판을 받았다. 이와 ...,City Event Planner,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
2,헝가리 왜 올림픽에서 보이콧 관련 있어요?,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...","1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",Sports Historian,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
3,2008년 하계 올림픽이 개최된 후 시청자 수에 어떤 변화가 있었는지 설명해 주시겠...,[베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독일 내에서만의 방송이...,베이징에서 열렸던 2008년 하계 올림픽의 시청자는 급격히 늘어났다.,City Event Planner,PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
4,"헬싱키 올림픽에서 소련 처음 참가했는데 그거 왜 중요해요, 그리고 그 전에는 소련 ...",[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...,소련은 헬싱키에서 열린 1952년 하계 올림픽 때 처음으로 참가했다. 그 전에는 소...,Sports Governance Researcher,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
5,"1936년 하계 올림픽이 정치적으로 어떻게 이용됐엇고, 그 결과는 뭐였나?",[<1-hop>\n\n쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체...,1936년 하계 올림픽에서 나치독일은 나치가 자비롭고 평화를 위한다는 점을 선전하고...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
6,"소련이 올림픽에서 보인 정치적 행동과 보이콧의 주요 원인은 무엇이며, 이러한 행동이...",[<1-hop>\n\n올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되...,"<1-hop>에 따르면, 소련은 1956년 하계 올림픽에서 헝가리 침공에 대한 항의...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
7,"1960년 동계 올림픽이 중계권을 특정 방송사에 최초로 판매한 올림픽이었다는 점과,...",[<1-hop>\n\n베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독...,1960년 동계 올림픽은 중계권을 어느 특정한 방송사에 판매한 첫 번째 올림픽이었다...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
8,2008년 하계 올림픽의 시청자 수가 급격히 늘어난 현상은 이전 올림픽들과 비교했을...,[<1-hop>\n\n베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독...,2008년 하계 올림픽의 시청자 수가 급격히 늘어난 현상은 이전 올림픽들과 비교했을...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
9,1952년 하계 올림픽에서 소련이 처음 참가한 것이 올림픽의 정치적 성격과 어떤 관...,[<1-hop>\n\n쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체...,1952년 하계 올림픽에서 소련이 처음 참가한 것은 올림픽이 정치 혹은 체제 선전의...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer


In [46]:
res = chain.invoke(eval_df['user_input'][0])

In [48]:
res['response']
res['retrieved_context']

['올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 상업성과 연계되어 이제 올림픽이 기존의 상업적인 스포츠쇼와 다를 게 없다는 것이다. 1996년과 2000년 하계 올림픽 기간 사이에는 올림픽 관련 상품 시장의 포화 상태가 일어나면서 IOC에게 또 다른 비판이 일었다. 개최도시가 올림픽 관련 물건들을 파려는 상인과 회사들로 넘쳐났던 것이다. IOC는 이에 대해 앞으로 다가올 올림픽에서는 과다 경쟁을 방지하고 이런 불미스라운 상황이 일어나지 않도록 대처하겠다고 응답했다. 또 다른 비판으로는, 개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있다는 점이다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 한다.',
 '올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최지 선정에는 약 2년이 걸린다. 유치를 희망하는 도시는 우선 자국의 올림픽 위원회에 신청을 해야 한다. 만약 한 국가에서 두 도시 이상이 유치를 희망한다면, 한 국가당 한 도시만 후보가 될 수 있다는 규칙에 따라 내부적으로 후보 도시를 결정해야 한다. 후보 도시가 결정되면 후보 도시가 소속된 국가의 올림픽 위원회는 IOC에 개최 신청을 하고, 신청 후에는 올림픽 개최에 대한 질의 응답서를 보내야 한다. 이 질의응답서에서 신청한 도시는 올림픽 헌장을 준수하며 IOC 상임이사회에 의한 다른 규정들을 지킬 것이라는 확신을 주어야 한다. 이 질의응답서는 전문가들이 검토하여 신청 도시들의 잠재성과 계획을 평가한다. 이 전문적인 평가를 바탕으로 IOC 상임이사회에서는 신청도시 중에서 후보도시를 고른다.  \n후보도시로 선택되면 그 도시들은 IOC에 보내는 후보도시에 관한 문서에 그들의 계획을

In [50]:
# Chain 응답들을 저장할 list
response_list = []
# Chain 이 반환한 context들을 지정할 list
retrieved_context_list = []

for user_input in eval_df['user_input']:
    res = chain.invoke(user_input)
    response_list.append(res['response'])
    retrieved_context_list.append(res['retrieved_context'])

In [51]:
print(len(response_list), len(retrieved_context_list))

10 10


In [52]:
response_list

['요약하면, 문맥에 따르면 구조적으로 IOC는 올림픽 브랜드·상징과 스폰서·중계권에 대한 수익을 통제하고 판매하는 반면, 개최국·개최도시는 대회 준비와 관련된 비용(경기장·인프라·조직비 등)을 부담하도록 되어 있기 때문입니다. 문맥의 근거를 정리하면 다음과 같습니다.\n\n- 올림픽 상징권과 스폰서·중계권 수입 통제: 문맥은 “IOC는 올림픽 상징으로 인한 수입뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다”고 명시하고 있습니다. 또한 TOP(올림픽 프로그램) 같은 제도를 통해 독점적 후원권을 판매하여 수익을 얻는다고 설명합니다(예: TOP 회원은 4년마다 5000만 달러를 지불).\n- 개최국이 비용을 부담하는 구조: 개최 희망 도시는 IOC에 신청하고, 개최에 필요한 자금 조달 능력을 입증해야 하며, 유치위원회가 IOC와 개최도시 계약서에 서명해야 공식 개최지가 됩니다. 즉 개최 준비와 자금 조달 책임은 개최도시/개최국에게 있습니다.\n- 역사적 배경과 상업화의 변화: 문맥은 과거에는 IOC가 상업적 연계를 거부했으나(브런디지 시대) 텔레비전 중계권 판매와 스폰서 계약 이후 재정 구조가 크게 달라졌고(예: LA 1984 조직위원회의 큰 이익, IOC 잔고 증가 등), 이로 인해 IOC의 재정 독립을 위한 상업화가 진행되었다고 기록하고 있습니다.\n- 결과적 비판과 위험: 따라서 비판의 핵심은 “개최국·개최도시는 모든 비용을 들이는데 IOC는 올림픽 상징으로부터 얻는 권리와 수입을 독차지하고 있다”는 점이며, 개최도시는 투자 대비 흑자 여부를 확신할 수 없는 상황에서도 유치와 준비를 해야 한다고 문맥에 나와 있습니다.\n\n이 때문에 개최국·개최도시는 막대한 비용과 재정 위험을 안고 준비를 진행하고, IOC는 상징권·스폰서·중계권 등에서 수익을 얻는 구조가 형성되어 있다는 설명이 문맥의 근거입니다.',
 '프랑코 정권이 무엇인지에 대해서는 주어진 문서에 설명이 없어 답을 할 수 없습니다. 정보가 부족해 답을 할 수없습니다.\n\n사마란치

In [53]:
retrieved_context_list

[['올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 상업성과 연계되어 이제 올림픽이 기존의 상업적인 스포츠쇼와 다를 게 없다는 것이다. 1996년과 2000년 하계 올림픽 기간 사이에는 올림픽 관련 상품 시장의 포화 상태가 일어나면서 IOC에게 또 다른 비판이 일었다. 개최도시가 올림픽 관련 물건들을 파려는 상인과 회사들로 넘쳐났던 것이다. IOC는 이에 대해 앞으로 다가올 올림픽에서는 과다 경쟁을 방지하고 이런 불미스라운 상황이 일어나지 않도록 대처하겠다고 응답했다. 또 다른 비판으로는, 개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있다는 점이다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 한다.',
  '올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최지 선정에는 약 2년이 걸린다. 유치를 희망하는 도시는 우선 자국의 올림픽 위원회에 신청을 해야 한다. 만약 한 국가에서 두 도시 이상이 유치를 희망한다면, 한 국가당 한 도시만 후보가 될 수 있다는 규칙에 따라 내부적으로 후보 도시를 결정해야 한다. 후보 도시가 결정되면 후보 도시가 소속된 국가의 올림픽 위원회는 IOC에 개최 신청을 하고, 신청 후에는 올림픽 개최에 대한 질의 응답서를 보내야 한다. 이 질의응답서에서 신청한 도시는 올림픽 헌장을 준수하며 IOC 상임이사회에 의한 다른 규정들을 지킬 것이라는 확신을 주어야 한다. 이 질의응답서는 전문가들이 검토하여 신청 도시들의 잠재성과 계획을 평가한다. 이 전문적인 평가를 바탕으로 IOC 상임이사회에서는 신청도시 중에서 후보도시를 고른다.  \n후보도시로 선택되면 그 도시들은 IOC에 보내는 후보도시에 관한 문서에 그들의 계

In [56]:
# eval_df 에 추가
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = retrieved_context_list
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name,response,retrieved_contexts
0,개최국이 올림픽 개최할 때 돈 많이 쓰고 IOC는 그냥 앉아서 올림픽 상징으로 돈 ...,[올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 ...,개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽...,Sports Historian,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer,"요약하면, 문맥에 따르면 구조적으로 IOC는 올림픽 브랜드·상징과 스폰서·중계권에 ...",[올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 ...
1,"프랑코 정권이 뭐였는지, 그리고 사마란치가 프랑코 정권에 협력한 게 왜 IOC에서 ...",[국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을...,사마란치 위원장은 스페인에서 프랑코 정권에 협력했다는 이유로 비판을 받았다. 이와 ...,City Event Planner,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer,프랑코 정권이 무엇인지에 대해서는 주어진 문서에 설명이 없어 답을 할 수 없습니다....,[국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을...
2,헝가리 왜 올림픽에서 보이콧 관련 있어요?,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...","1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",Sports Historian,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer,1956년 소련의 헝가리 침공에 항의하여 네덜란드·스페인·스위스 등이 1956년 하...,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스..."
3,2008년 하계 올림픽이 개최된 후 시청자 수에 어떤 변화가 있었는지 설명해 주시겠...,[베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독일 내에서만의 방송이...,베이징에서 열렸던 2008년 하계 올림픽의 시청자는 급격히 늘어났다.,City Event Planner,PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer,정보에 따르면 2008년 베이징 하계 올림픽에서는 시청자 수가 급격히 늘어났습니다....,[베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독일 내에서만의 방송이...
4,"헬싱키 올림픽에서 소련 처음 참가했는데 그거 왜 중요해요, 그리고 그 전에는 소련 ...",[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...,소련은 헬싱키에서 열린 1952년 하계 올림픽 때 처음으로 참가했다. 그 전에는 소...,Sports Governance Researcher,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer,컨텍스트에 따르면:\n\n- 중요성: 소련이 1952년 헬싱키 하계올림픽에 처음 참...,[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...


In [ ]:
################################################################
# EvaluationDataset을 생성 -> RAGAS 평가 데이터셋 타입
################################################################
from ragas import EvaluationDataset
# from_xxxx() xxxx 타입의 객체를 EvaluationDataset객체로 변환.
eval_dataset = EvaluationDataset.from_pandas(
    eval_df[["user_input", "retrieved_contexts", "response", "reference"]]
)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=10)

In [ ]:
##########################
# 평가
##########################
from ragas.metrics import (
    LLMContextRecall, # Context Recall 평가함수
    LLMContextPrecisionWithReference, # Context Precision
    Faithfulness, 
    AnswerRelevancy
)

from ragas import evaluate

eval_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))
# 평가할 함수들을 List로 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# 평가를 진행 - evaluate() 함수 이용
eval_result = evaluate(dataset=eval_dataset, metrics=metrics)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_23072\1589191573.py:4: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import (
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23072\1589191573.py:4: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextPrecisionWithReference
  from ragas.metrics import (
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23072\1589191573.py:4: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import (
C:\User

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


In [62]:
print(type(eval_result))
eval_result

<class 'ragas.dataset_schema.EvaluationResult'>


{'context_recall': 0.9500, 'llm_context_precision_with_reference': 0.9583, 'faithfulness': 0.9500, 'answer_relevancy': 0.6376}

In [ ]:
# 개별 Point에 대한 평가 결과.
result_df = eval_result.to_pandas()
result_df 

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,개최국이 올림픽 개최할 때 돈 많이 쓰고 IOC는 그냥 앉아서 올림픽 상징으로 돈 ...,[올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 ...,"요약하면, 문맥에 따르면 구조적으로 IOC는 올림픽 브랜드·상징과 스폰서·중계권에 ...",개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽...,1.0,0.833333,1.000,0.706527
1,"프랑코 정권이 뭐였는지, 그리고 사마란치가 프랑코 정권에 협력한 게 왜 IOC에서 ...",[국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을...,프랑코 정권이 무엇인지에 대해서는 주어진 문서에 설명이 없어 답을 할 수 없습니다....,사마란치 위원장은 스페인에서 프랑코 정권에 협력했다는 이유로 비판을 받았다. 이와 ...,0.5,1.000000,1.000,0.000000
2,헝가리 왜 올림픽에서 보이콧 관련 있어요?,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...",1956년 소련의 헝가리 침공에 항의하여 네덜란드·스페인·스위스 등이 1956년 하...,"1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",1.0,1.000000,1.000,0.788028
3,2008년 하계 올림픽이 개최된 후 시청자 수에 어떤 변화가 있었는지 설명해 주시겠...,[베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독일 내에서만의 방송이...,정보에 따르면 2008년 베이징 하계 올림픽에서는 시청자 수가 급격히 늘어났습니다....,베이징에서 열렸던 2008년 하계 올림픽의 시청자는 급격히 늘어났다.,1.0,0.750000,1.000,0.629736
4,"헬싱키 올림픽에서 소련 처음 참가했는데 그거 왜 중요해요, 그리고 그 전에는 소련 ...",[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...,컨텍스트에 따르면:\n\n- 중요성: 소련이 1952년 헬싱키 하계올림픽에 처음 참...,소련은 헬싱키에서 열린 1952년 하계 올림픽 때 처음으로 참가했다. 그 전에는 소...,1.0,1.000000,0.875,0.703424
5,"1936년 하계 올림픽이 정치적으로 어떻게 이용됐엇고, 그 결과는 뭐였나?",[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...,나치 독일은 1936년 베를린 올림픽을 자국을 자비롭고 평화를 지향하는 나라로 보이...,1936년 하계 올림픽에서 나치독일은 나치가 자비롭고 평화를 위한다는 점을 선전하고...,1.0,1.000000,1.000,0.580687
6,"소련이 올림픽에서 보인 정치적 행동과 보이콧의 주요 원인은 무엇이며, 이러한 행동이...","[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...",정보는 다음과 같이 정리됩니다.\n\n- 소련 관련 정치적 행동과 보이콧의 사례(컨...,"<1-hop>에 따르면, 소련은 1956년 하계 올림픽에서 헝가리 침공에 대한 항의...",1.0,1.000000,1.000,0.646191
7,"1960년 동계 올림픽이 중계권을 특정 방송사에 최초로 판매한 올림픽이었다는 점과,...",[베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독일 내에서만의 방송이...,1960년 동계 올림픽이 중계권을 특정 방송사에 최초로 판매한 사건은 올림픽의 전파...,1960년 동계 올림픽은 중계권을 어느 특정한 방송사에 판매한 첫 번째 올림픽이었다...,1.0,1.000000,1.000,0.798112
8,2008년 하계 올림픽의 시청자 수가 급격히 늘어난 현상은 이전 올림픽들과 비교했을...,[베를린에서 열렸던 1936년 하계 올림픽은 전 세계가 아닌 독일 내에서만의 방송이...,문맥에 따르면 2008년 베이징 하계올림픽의 시청자 수 급증은 ‘텔레비전의 영향력 ...,2008년 하계 올림픽의 시청자 수가 급격히 늘어난 현상은 이전 올림픽들과 비교했을...,1.0,1.000000,1.000,0.765572
9,1952년 하계 올림픽에서 소련이 처음 참가한 것이 올림픽의 정치적 성격과 어떤 관...,[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...,Context에 따르면 소련은 1952년 헬싱키 대회에 처음으로 올림픽에 참가하기 ...,1952년 하계 올림픽에서 소련이 처음 참가한 것은 올림픽이 정치 혹은 체제 선전의...,1.0,1.000000,0.625,0.758205
